# Raymarching Visualization

In [1]:
# Raymarching illustration in SageMath 3D
# Using vector() and Sage's plotting utilities

from sage.all import *
from sage.plot.plot3d.shapes2 import sphere
from sage.plot.plot3d.shapes import arrow3d

# ---------- Scene parameters ----------
eye = vector([0, -3, 0])       # camera/eye position
img_y = 0.0                    # y-position of the image plane
uv_min, uv_max = -1.2, 1.2     # horizontal UV range
width = 21                     # number of pixels across
max_steps = 100
max_dist = 6.0
hit_eps = 1e-3

# Sphere geometry
sphere_center = vector([0.2, 0.9, 0])
sphere_radius = 0.8

# ---------- SDF Functions ----------
def sdf_sphere(p, center=sphere_center, r=sphere_radius):
    """Signed distance to a sphere (2D XY, Z fixed at 0)."""
    return (p - center).norm() - r

def scene_sdf(p):
    return sdf_sphere(p)

def normalize(v):
    n = v.norm()
    return v / n if n > 0 else v

# ---------- Raymarching ----------
uv_xs = srange(uv_min, uv_max, (uv_max-uv_min)/(width-1))

# Start with an empty scene
plot_obj = sphere(center=eye, size=0.0, color='white', opacity=0)  # invisible placeholder

# Add sphere object
plot_obj += sphere(center=sphere_center, size=sphere_radius, color="lightblue", opacity=0.4)

# Add image plane as a transparent rectangle
plane_points = [
    (uv_min, img_y, 0),
    (uv_max, img_y, 0),
    (uv_max, img_y, 0.01),
    (uv_min, img_y, 0.01)
]
plot_obj += polygon3d(plane_points, color="lightgray", opacity=0.2)

# Add eye point
plot_obj += sphere(center=eye, size=0.05, color="black")

for ux in uv_xs:
    target = vector([ux, img_y, 0])
    direction = normalize(target - eye)
    t = 0.0
    hit = False
    for step in range(max_steps):
        p = eye + direction * t
        d = scene_sdf(p)
        if d < hit_eps:
            hit = True
            break
        t += d
        if t >= max_dist:
            break
    endpoint = eye + direction * min(t, max_dist)
    
    # Arrow for ray
    color = "green" if hit else "gray"
    plot_obj += arrow3d(eye, endpoint, color=color, width=0.01)
    
    # Red dot at hit point
    if hit:
        plot_obj += sphere(center=endpoint, size=0.03, color="red")

# Show plot
plot_obj.show(frame=False, aspect_ratio=1)


Graphics3d Object

In [30]:
from sage.all import *
from sage.plot.plot3d.shapes2 import sphere
from sage.plot.plot3d.shapes import arrow3d

# ---------- SDF Functions ----------
def sdf_sphere(p, center=sphere_center, r=sphere_radius):
    return (p - center).norm() - r

def scene_sdf(p):
    return sdf_sphere(p)

def normalize(v):
    n = v.norm()
    return v / n if n > 0 else v

@interact
def render(ball_hit=False, ball_opacity=[0.0, 0.1, .. 1.0]):
    # ---------- Scene parameters ----------
    eye = vector([0, -4, 0])       # camera position
    img_y = 0.0                    # image plane y-position
    uv_min, uv_max = -1.0, 1.0     # horizontal UV range
    res_x, res_z = 5, 5            # pixel resolution (small for clarity)
    max_steps = 10
    max_dist = 6.0
    hit_eps = 1e-3
    
    # Sphere geometry
    sphere_center = vector([0.0, 1.2, 0.0])
    sphere_radius = 0.8

    # ---------- Raymarching visualization ----------
    plot_obj = sphere(center=eye, size=1.0, color='white', opacity=0)  # start empty
    
    # Add sphere object
    plot_obj += sphere(center=sphere_center, size=sphere_radius, color="lightblue", opacity=1.0)

    # Add image plane as a transparent grid
    pixel_width = (uv_max - uv_min) / res_x
    pixel_height = (uv_max - uv_min) / res_z
    for ix in range(res_x):
        for iz in range(res_z):
            x0 = uv_min + ix * pixel_width
            z0 = uv_min + iz * pixel_height
            rect = [
                (x0, img_y, z0),
                (x0+pixel_width, img_y, z0),
                (x0+pixel_width, img_y, z0+pixel_height),
                (x0, img_y, z0+pixel_height)
            ]
            plot_obj += polygon3d(rect, color="lightgray", opacity=0.5)
    
    # Add eye point
    plot_obj += sphere(center=eye, size=0.05, color="black")
    
    # Loop over pixels
    for ix in range(res_x):
        for iz in range(res_z):
            # Pixel center
            px = uv_min + (ix + 0.5) * pixel_width
            pz = uv_min + (iz + 0.5) * pixel_height
            target = vector([px, img_y, pz])
            
            direction = normalize(target - eye)
            t = 0.0
            hit = False
            
            # Marching steps
            for step in range(max_steps):
                p = eye + direction * t
                d = scene_sdf(p)
                
                # Draw step sphere to visualize SDF radius
                plot_obj += sphere(center=p, size=d if t > hit_eps else 0.00, color="yellow", opacity=ball_opacity if ball_hit else 0.00)
                
                if d < hit_eps:
                    hit = True
                    break
                t += d
                if t >= max_dist:
                    break
            
            endpoint = eye + direction * min(t, max_dist)
            color = "green" if hit else "gray"
            plot_obj += arrow3d(eye, endpoint, color=color, width=0.005, radius=0.01)
            
            if hit:
                plot_obj += sphere(center=endpoint, size=0.03, color="red")
    
    # Show plot
    plot_obj.show(frame=True, aspect_ratio=1)


Interactive function <function render at 0x16b8eb380> with 2 widgets
  ball_hit: Checkbox(value=False, description='ball_hit')
  ball_opacity: Dropdown(description='ball_opacity', options=(0.000000000000000, 0.100000000000000, 0.200000000000000, 0.300000000000000, 0.400000000000000, 0.500000000000000, 0.600000000000000, 0.700000000000000, 0.800000000000000, 0.900000000000000, 1.00000000000000), value=0.000000000000000)